In [3]:
import pandas as pd
import pickle 
from pathlib import Path
import numpy as np
import sklearn 

 # machine learning libraries
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier
import xgboost as xgb

#Visualize Features
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
import seaborn as sns


In [16]:
uncontained_df = pd.read_pickle('uncontained_names.pkl')
df_cand = pd.read_pickle(Path('.')/'..'/'df_cand.pkl')
df_comm = pd.read_pickle(Path('.')/'..'/'committee_id_formatted_v1.pkl')


,name,candidate_id,state,incumbent_challenge_full,party_full,office_full,cycles,election_years,last_file_date,load_date
0,aalders timothy noel,S2UT00229,UT,Open seat,CONSTITUTION PARTY,Senate,2012,"[2012, 2018]",2018-04-23,2018-08-01T00:13:22
1,aalders timothy noel,S2UT00229,UT,Open seat,CONSTITUTION PARTY,Senate,2014,"[2012, 2018]",2018-04-23,2018-08-01T00:13:22
2,aalders timothy noel,S2UT00229,UT,Open seat,CONSTITUTION PARTY,Senate,2016,"[2012, 2018]",2018-04-23,2018-08-01T00:13:22
3,aalders timothy noel,S2UT00229,UT,Open seat,CONSTITUTION PARTY,Senate,2018,"[2012, 2018]",2018-04-23,2018-08-01T00:13:22
4,aanestad samuel,H2CA01110,CA,Challenger,REPUBLICAN PARTY,House,2012,[2012],2012-02-22,2013-04-26T09:04:30


In [66]:
uncontained_df


,0,1,2,3,4
0,kanjorski,2010,PA,"[barletta, kanjorski]",0
1,etheridge,2010,NC,"[ellmers, etheridge]",0
2,boucher,2010,VA,"[boucher, griffith]",0
3,bass,2010,NH,"[bass, kuster]",1
4,rahall,2010,WV,"[rahall, maynard]",1
5,fitzpatrick,2010,PA,"[fitzpatrick, murphy]",1
6,fitzpatrick,2010,PA,"[fitzpatrick, murphy]",1
7,dingell,2010,MI,"[dingell, steele]",1
8,reid,2010,NV,"[reid, angle]",1
9,leahy,2010,VT,"[leahy, britton]",1


In [34]:
df_cand.head()


,name,candidate_id,state,incumbent_challenge_full,party_full,office_full,cycles,election_years,last_file_date,load_date
0,aalders timothy noel,S2UT00229,UT,Open seat,CONSTITUTION PARTY,Senate,2012,"[2012, 2018]",2018-04-23,2018-08-01T00:13:22
1,aalders timothy noel,S2UT00229,UT,Open seat,CONSTITUTION PARTY,Senate,2014,"[2012, 2018]",2018-04-23,2018-08-01T00:13:22
2,aalders timothy noel,S2UT00229,UT,Open seat,CONSTITUTION PARTY,Senate,2016,"[2012, 2018]",2018-04-23,2018-08-01T00:13:22
3,aalders timothy noel,S2UT00229,UT,Open seat,CONSTITUTION PARTY,Senate,2018,"[2012, 2018]",2018-04-23,2018-08-01T00:13:22
4,aanestad samuel,H2CA01110,CA,Challenger,REPUBLICAN PARTY,House,2012,[2012],2012-02-22,2013-04-26T09:04:30


In [ ]:

def reformat_by_cycle(df):
    df_temp = df.copy()
    
    # Create the DataFrame with 54446 rows and columns -- all NaNs
    cols = df_temp.columns
    df_cand = pd.DataFrame(np.full((len_reform_df, len(cols)), np.nan), columns=cols)
    df_cand['cycles'] = df_cand['cycles'].astype(list)
    df_cand['election_years'] = df_cand['election_years'].astype(list)
    
    i = 0
    j= 0
    for list_cycles in df_temp.cycles:
        for cycle in list_cycles:
            df_cand.iloc[j,:] = df_temp.iloc[i,:] 
            df_cand.iloc[j, 6] = cycle
            j+=1
    
        i+=1
    return df_cand

In [71]:
df_temp = df_cand.copy()

cols = df_temp.columns
# df_cand_matched = pd.DataFrame(np.full((1000, len(cols)), np.nan), columns=cols)
df_cand_matched = pd.DataFrame(columns=cols)
df_cand_matched['election_years'] = df_cand_matched['election_years'].astype(list)

j = 0

for index, row in uncontained_df.iterrows():
#   & (df_cand.cycles == row[1])
    test = df_temp[(df_temp.name.str.contains(row[0])) & (df_temp.state == row[2]) & (df_temp.cycles == int(row[1]))]
    df_cand_matched = pd.concat((df_cand_matched, test))

df_cand_matched


,name,candidate_id,state,incumbent_challenge_full,party_full,office_full,cycles,election_years,last_file_date,load_date
15039,etheridge bob,H6NC02080,NC,None,DEMOCRATIC PARTY,House,2010,"[1996, 1998, 2000, 2002, 2004, 2006, 2008, 2010]",2010-11-19,2011-02-24T10:53:11
4955,boucher frederick c,H2VA09010,VA,None,DEMOCRATIC PARTY,House,2010,"[1982, 1984, 1986, 1988, 1990, 1992, 1994, 199...",2009-11-23,2011-02-24T10:53:11
16060,fitzpatrick michael g.,H4PA08074,PA,None,REPUBLICAN PARTY,House,2010,"[2004, 2006, 2010, 2012, 2014, 2016]",2014-07-15,2015-01-02T21:41:16
16060,fitzpatrick michael g.,H4PA08074,PA,None,REPUBLICAN PARTY,House,2010,"[2004, 2006, 2010, 2012, 2014, 2016]",2014-07-15,2015-01-02T21:41:16
30010,lujan grisham michelle,H8NM01257,NM,Incumbent,DEMOCRATIC PARTY,House,2012,"[2008, 2012, 2014, 2016, 2018]",2016-11-17,2017-02-27T21:01:34


# Functions to Collect Donor Information for Candidates

In [11]:
# Use by_state/by_candidate, by_size/by_candidate
# QUESTION, Does this by_candidate search include donations made to committee's? 

# Use /schedules/schedule_a/by_size/by_candidate/
# Use /schedules/schedule_a/by_state/by_candidate/


# by_size/by_candidate
#  -0    $200 and under
#  -200  $200.01 - $499.99
#  -500  $500 - $999.99
#  -1000 $1000 - $1999.99
#  -2000 $2000 +

def cand_donor_size(candidate_id, cycle): 
    # Returns aggregated totals of contribution amounts that fit within defined groupings

    url_size = 'https://api.open.fec.gov/v1/schedules/schedule_a/\
by_size/by_candidate/\
?per_page=100\
&page=1\
&sort_null_only=false\
&election_full=false\
&cycle={}\
&sort_hide_null=false\
&api_key={}\
&candidate_id={}'.format(cycle, api_key, candidate_id)

    donor_cand_size = requests.get(url_size)
#     print(donor_cand_size.status_code)
    if donor_cand_size.status_code != 200:
        return pd.DataFrame(['time_out'])
    donor_size_result = donor_cand_size.json()['results']
    df_dcand_size = pd.DataFrame(donor_size_result)
    return df_dcand_size

    
    
# Schedule A individual receipts aggregated by contributor state.
# This is an aggregate of only individual contributions. 
def cand_donor_state(candidate_id, cycle): 
    # Returns aggregated totals of contribution amounts that fit within defined groupings
    url_state = 'https://api.open.fec.gov/v1/schedules/schedule_a/\
by_state/by_candidate/\
?per_page=100\
&page=1\
&sort_null_only=false\
&election_full=false\
&cycle={}\
&sort_hide_null=false\
&api_key={}\
&candidate_id={}'.format(cycle, api_key, candidate_id)
    
    donor_cand_state = requests.get(url_state)
    #print(donor_cand_state.status_code)
    if donor_cand_state.status_code != 200:
        return pd.DataFrame(['time_out'])
    donor_state_result = donor_cand_state.json()['results']
    df_dcand_state = pd.DataFrame(donor_state_result)
    return df_dcand_state



In [12]:
# Use /committee/{committee_id}/schedules/schedule_a/by_size/
# Use /committee/{committee_id}/schedules/schedule_a/by_state/
# Extract candidate_ids supported by each committee, use DataFrame B

def comm_donor_size(committee_id):
    url = 'https://api.open.fec.gov/v1/committee/\
{}/schedules/schedule_a/\
by_size/?sort_hide_null=false\
&per_page=100\
&page=1\
&api_key={}\
&sort_null_only=false'.format(committee_id, api_key)
    
    donor_comm_size = requests.get(url)
    #print(donor_comm_size.status_code)
    if donor_comm_size.status_code != 200:
        return pd.DataFrame(['time_out'])
    donor_comm_size = donor_comm_size.json()['results']
    df_dcomm_size = pd.DataFrame(donor_comm_size)
    return df_dcomm_size

def comm_donor_state(committee_id):
    url = 'https://api.open.fec.gov/v1/committee/\
{}/schedules/schedule_a/\
by_state/?sort_hide_null=false\
&per_page=100\
&page=1\
&api_key={}\
&sort_null_only=false'.format(committee_id, api_key)
    
    donor_comm_state = requests.get(url)
    #print(donor_comm_state.status_code)
    if donor_comm_state.status_code != 200:
        return pd.DataFrame(['time_out'])
    donor_comm_state = donor_comm_state.json()['results']
    df_dcomm_state = pd.DataFrame(donor_comm_state)
    return df_dcomm_state



In [13]:
def merge_two_dicts(x, y):
    z = x.copy()   # start with x's keys and values
    z.update(y)    # modifies z with y's keys and values & returns None
    return z

a = cand_donor_state('P80001571', 2016)
states = a.state_full

In [15]:
import time
# Input: Candidate Name, Election Type, Date-time of election (Use Pandas Date time Format)
# 1. Will incorporate a funcation call to 
# 2. to retrieve the fraction of committee donations for that candidate

# by_size/by_candidate
#  -0    $200 and under Level 1 
#  -200  $200.01 - $499.99 Level 2 
#  -500  $500 - $999.99 Level 3 
#  -1000 $1000 - $1999.99 Level 4
#  -2000 $2000 + Level 5

# Election Result: 
#     0 Lose
#     1 Win
#     3 No Data




# Returns rows, 1 row per cycle with all features for one candidate. 
def candidate_donor_df(candidate_name, states, df_candidate_id):
    cand_row = df_candidate_id[df_candidate_id.name == candidate_name]
    cand_ids = cand_row.candidate_id.values
    
    j = 0
    
    #Initialize dataframe
    cand_info = ['name', 'cand_id', 'state', 'cycle', 'incumbent', 'office_full',
                 'party', 'committee_id', 'committee_name','race_candidates', 'election_result']
    donation_levels = ['Donation Level 1',
               'Donation Level 2', 
               'Donation Level 3',
               'Donation Level 4',
               'Donation Level 5']
    levels = [0, 200, 500, 1000, 2000]
    cols = np.append(cand_info, states)
    cols = np.append(cols, donation_levels)

    df = pd.DataFrame(np.full((1000, len(cols)), np.nan), columns=cols)
    cand_id = cand_ids[0]
#     for cand_id in cand_ids:
    #print(cand_id)

    cycles = int(cand_row.cycles.values[0])




    #Iterate through candidates cycles, add all cycles as new rows. Append to each column each

    #print(cycles)
#         for cycle in cycles:
    df_size = cand_donor_size(cand_id, cycles)
    df_state = cand_donor_state(cand_id, cycles)

    if len(df_size) > 0 or len(df_state) > 0:
        
        if df_size.values.size == 0 or df_state.values.size == 0:
                return df.dropna()
                
        if df_size.iloc[0,0] == 'time_out' or df_state.iloc[0,0] == 'time_out':
            print('API Time Out, Sleep for 1 hour')
            time.sleep(3610)
            df_size = cand_donor_size(cand_id, cycles)
            df_state = cand_donor_state(cand_id, cycles)
            

    if df_size.values.size == 0 or df_state.values.size == 0:
        return df.dropna()
    donor_states = df_state.state_full

    #Build row of information (donor_states)

    # Idea
    # Create dictionary with states as keys and initialized with 0s as values. 
    initialize_array = np.zeros(len(states))
    state_dict = dict(zip(states, initialize_array))

    for ds in donor_states:            
        # Overwrite select values with information from df_state
        state_dict[ds] = df_state[df_state.state_full == ds].total.values[0]


    # Add size features to state_dict 


    level_array = np.zeros(5)
    level_dict = dict(zip(donation_levels, level_array))

    for level in df_size['size']:
        if level == 0:
            donation_level = 'Donation Level 1'
        if level == 200:
            donation_level = 'Donation Level 2'
        if level == 500:
            donation_level = 'Donation Level 3'
        if level == 1000:
            donation_level = 'Donation Level 4'
        if level == 2000:
            donation_level = 'Donation Level 5'
        level_dict[donation_level] = df_size[df_size['size'] == level].total.values[0]


    cand_info_dict = {'name': candidate_name, 
                      'cand_id': cand_id, 
                      'state': cand_row.state.values[0],
                      'cycle': cycles,
                      'incumbent': cand_row.incumbent_challenge_full.values[0],
                      'office_full': cand_row.office_full.values[0],
                      'party': cand_row.party_full.values[0],
                      'committee_id': 0, 
                      'committee_name': 0,
                      'race_candidates': cand_row.race_candidates.values[0],
                      'election_result': cand_row.election_result.values[0]}
    state_dict = merge_two_dicts(cand_info_dict, state_dict)
    state_dict = merge_two_dicts(state_dict, level_dict)
    #return state_dict
    #df.iloc[j]
    #print(np.array(list(state_dict.values())))
    temp = list(state_dict.values())
    df.iloc[j] = np.array(temp, dtype=object) # An array of the row information
    j += 1
    return df.dropna()

                         
                         
        

In [19]:
import time
#Add df_candidate_id input to the previous function for candidate donor information collection function 

def get_committee_candidates(candidate_name, states, df_candidate_id, df_comm):
    cand_row = df_candidate_id[df_candidate_id.name == candidate_name]
    cand_ids = cand_row.candidate_id.values
    j = 0
    
    #Initialize dataframe
    cand_info = ['name', 'cand_id', 'state', 'cycle', 'incumbent', 'office_full', 'party', 
                 'committee_id', 'committee_name', 'race_candidates', 'election_result']
    donation_levels = ['Donation Level 1',
               'Donation Level 2', 
               'Donation Level 3',
               'Donation Level 4',
               'Donation Level 5']
    levels = [0, 200, 500, 1000, 2000]
    cols = np.append(cand_info, states)
    cols = np.append(cols, donation_levels)

    df = pd.DataFrame(np.full((1000, len(cols)), np.nan), columns=cols)
    cand_id = cand_ids[0]
#     for cand_id in cand_ids:
    comm_rows = df_comm[df_comm.candidate_ids == cand_id]
    #return comm_rows

    #for row in comm_rows: 
    for index, row in comm_rows.iterrows(): #Iterate through each row of the dataframe comm_row
        #print('iter')
        candidate_donor_weight = row.candidate_donor_weight
        cycles = row.cycles
        comm_id = row.committee_id
        #Iterate through candidates cycles, add all cycles as new rows. Append to each column each
        #print(cycles)
#         cycle = cycles[0]
        for cycle in cycles:
            df_size = comm_donor_size(comm_id)
            df_state = comm_donor_state(comm_id)

            
            if len(df_size) > 0 or len(df_state) > 0:
                
                if df_size.values.size == 0 or df_state.values.size == 0:
                    break
                
                if df_size.iloc[0,0] == 'time_out' or df_state.iloc[0,0] == 'time_out':
                    print('API Time Out, Sleep for 1 hour')
                    time.sleep(3610)
                    df_size = comm_donor_size(comm_id)
                    df_state = comm_donor_state(comm_id)

            if df_size.values.size == 0 or df_state.values.size == 0:
                break
            donor_states = df_state.state_full

            #Build row of information (donor_states)

            # Idea
            # Create dictionary with states as keys and initialized with 0s as values. 
            initialize_array = np.zeros(len(states))
            state_dict = dict(zip(states, initialize_array))

            for ds in donor_states:            
                # Overwrite select values with information from df_state
                state_row = df_state[df_state.state_full == ds]
                state_dict[ds] = state_row.total.values[0] * candidate_donor_weight


            # Add size features to state_dict 


            level_array = np.zeros(5)
            level_dict = dict(zip(donation_levels, level_array))

            for level in df_size['size']:
                if level == 0:
                    donation_level = 'Donation Level 1'
                if level == 200:
                    donation_level = 'Donation Level 2'
                if level == 500:
                    donation_level = 'Donation Level 3'
                if level == 1000:
                    donation_level = 'Donation Level 4'
                if level == 2000:
                    donation_level = 'Donation Level 5'
                size_row = df_size[df_size['size'] == level]
                level_dict[donation_level] = size_row.total.values[0] * candidate_donor_weight


            cand_info_dict = {'name': candidate_name, 
                              'cand_id': cand_id, 
                              'state': row.state,
                              'cycle': cycle,
                              'incumbent': 0,
                              'office_full': 0,
                              'party': 0,
                              'committee_id': row.committee_id,
                              'committee_name': row[11],
                              'race_candidates': 0,
                              'election_result': 0}
            state_dict = merge_two_dicts(cand_info_dict, state_dict)
            state_dict = merge_two_dicts(state_dict, level_dict)

            #return state_dict


            #return state_dict.values()
            df.iloc[j] = np.array(list(state_dict.values()), dtype = object) # An array of the row information
            j += 1
            break
    return df.dropna()
    
